In [1]:
from lsst.rsp import get_tap_service
import sqlite3
import pandas as pd

In [2]:
service = get_tap_service("ssotap")

In [3]:
db_fname = "adler_demo_testing_database.db"

In [4]:
cnx = sqlite3.connect(db_fname)

In [5]:
# ssoid = 8268570668335894776
ssoid_list = (8268570668335894776, 2150553186630, 3369984299447, 5992863104062, 6098332225018)

test_id = 6098332225018
fake_id = 6098332225018000

In [6]:
filter_name = "r"

In [7]:
# diasource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.diaSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
diasource_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.diaSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [8]:
diatable = service.search(diasource_sql_query).to_table().to_pandas()

In [9]:
# sssource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.SSSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
sssource_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.SSSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [10]:
sssource_table = service.search(sssource_sql_query).to_table().to_pandas()

In [11]:
sssource_table.columns

Index(['diaSourceId', 'eclipticBeta', 'eclipticLambda', 'galacticB',
       'galacticL', 'heliocentricDist', 'heliocentricVX', 'heliocentricVY',
       'heliocentricVZ', 'heliocentricX', 'heliocentricY', 'heliocentricZ',
       'mpcUniqueId', 'phaseAngle', 'predictedDecErr', 'predictedMagnitude',
       'predictedMagnitudeErr', 'predictedRaDecCov', 'predictedRaErr',
       'residualDec', 'residualRa', 'ssObjectId', 'topocentricDist',
       'topocentricVX', 'topocentricVY', 'topocentricVZ', 'topocentricX',
       'topocentricY', 'topocentricZ'],
      dtype='object')

In [12]:
# ssobject_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.SSObject
#             WHERE
#                 ssObjectId = {ssoid}
#             """
ssobject_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.SSObject
            WHERE
                ssObjectId in {ssoid_list}
            """

In [13]:
ssobject_table = service.search(ssobject_sql_query).to_table().to_pandas()

In [14]:
ssobject_table.columns

Index(['arc', 'discoverySubmissionDate', 'firstObservationDate', 'flags',
       'g_Chi2', 'g_G12', 'g_G12Err', 'g_H', 'g_H_gG12_Cov', 'g_HErr',
       'g_Ndata', 'i_Chi2', 'i_G12', 'i_G12Err', 'i_H', 'i_H_iG12_Cov',
       'i_HErr', 'i_Ndata', 'maxExtendedness', 'medianExtendedness',
       'minExtendedness', 'MOID', 'MOIDDeltaV', 'MOIDEclipticLongitude',
       'MOIDTrueAnomaly', 'numObs', 'r_Chi2', 'r_G12', 'r_G12Err', 'r_H',
       'r_H_rG12_Cov', 'r_HErr', 'r_Ndata', 'ssObjectId', 'u_Chi2', 'u_G12',
       'u_G12Err', 'u_H', 'u_H_uG12_Cov', 'u_HErr', 'u_Ndata', 'y_Chi2',
       'y_G12', 'y_G12Err', 'y_H', 'y_H_yG12_Cov', 'y_HErr', 'y_Ndata',
       'z_Chi2', 'z_G12', 'z_G12Err', 'z_H', 'z_H_zG12_Cov', 'z_HErr',
       'z_Ndata'],
      dtype='object')

In [15]:
ssobject_table["r_Ndata"]

0     99
1     34
2     32
3    109
4     38
Name: r_Ndata, dtype: int32

In [16]:
# mpcorb_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.MPCORB
#             WHERE
#                 ssObjectId = {ssoid}
#             """
mpcorb_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.MPCORB
            WHERE
                ssObjectId in {ssoid_list}
            """

In [17]:
mpcorb_table = service.search(mpcorb_sql_query).to_table().to_pandas()

In [18]:
mpcorb_table

,arc,arcEnd,arcStart,computer,e,epoch,flags,fullDesignation,incl,lastIncludedObservation,...,nopp,peri,pertsLong,pertsShort,q,reference,rms,ssObjectId,tperi,uncertaintyParameter
0,NaN,,,,0.095122,60065.0,0,2011 2012 AA35,5.07716,NaN,...,0,17.125305,,,2.037489,,NaN,2150553186630,59579.605855,
1,NaN,,,,0.209602,60065.0,0,2011 S100h4WYa,7.78333,NaN,...,0,136.833063,,,1.842686,,NaN,3369984299447,59826.524077,
2,NaN,,,,0.090991,60065.0,0,2011 S100uH5sa,12.13587,NaN,...,0,289.224457,,,2.720239,,NaN,5992863104062,60859.148369,
3,NaN,,,,0.139678,60065.0,0,2011 2007 YP19,4.44057,NaN,...,0,201.500952,,,2.721865,,NaN,6098332225018,60673.714400,
4,NaN,,,,0.716881,60065.0,0,2011 2014 QL433,4.42757,NaN,...,0,260.546820,,,0.589829,,NaN,8268570668335894776,60112.792837,


In [19]:
# add a new fake object

In [20]:
diatable

,band,ccdVisitId,dec,decErr,decTrue,diaObjectId,diaSourceId,mag,magErr,magTrueVband,midPointMjdTai,nameTrue,ra,ra_dec_Cov,raErr,raTrue,snr,ssObjectId,ssObjectReassocTime
0,r,826978,-22.958176,0.000005,-22.958163,2905100135900213485,18188774906504369,22.122999,0.041,22.033504,61648.28255,2012 AA35,323.890681,0.0,0.000005,323.890684,22.464245,2150553186630,61648.28255
1,r,2035180,-14.553622,0.000018,-14.553617,6644478959102540493,87006313523607576,23.186001,0.176,23.277239,63793.11123,2012 AA35,220.572183,0.0,0.000018,220.572151,5.233148,2150553186630,63793.11123
2,i,538202,1.090052,0.000003,1.090046,-1814395187774575115,102043887374547575,21.809999,0.037,21.941309,61155.17906,2012 AA35,195.195758,0.0,0.000003,195.195759,24.892813,2150553186630,61155.17906
3,g,1994507,-14.133374,0.000004,-14.133380,-2764074387554066817,149948551075835426,22.665001,0.049,21.961533,63715.30357,2012 AA35,230.760251,0.0,0.000004,230.760246,18.796614,2150553186630,63715.30357
4,g,534253,0.589982,0.000003,0.589986,-2101259887024488987,186636841932196305,22.417000,0.028,21.665754,61145.18953,2012 AA35,197.511556,0.0,0.000003,197.511556,32.894073,2150553186630,61145.18953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,g,7799,-10.670760,0.000004,-10.670763,-874443130836536444,-1345853232552594655,22.945000,0.046,22.346977,60228.05606,2014 QL433,323.532914,0.0,0.000004,323.532914,20.022480,8268570668335894776,60228.05606
846,r,1090090,29.320428,0.000035,29.320468,-7987737159207644782,-1334792330958883646,23.068001,0.211,23.155615,62113.17449,2014 QL433,74.691787,0.0,0.000035,74.691838,4.365090,8268570668335894776,62113.17449
847,r,4492,-10.889507,0.000006,-10.889517,-5709489065863214846,-786563943595319518,22.143000,0.057,22.113560,60223.98877,2014 QL433,322.778901,0.0,0.000006,322.778904,16.158491,8268570668335894776,60223.98877
848,i,1237285,-18.625892,0.000008,-18.625884,-713349703496633846,-754458044377129801,21.313000,0.060,21.484858,62406.11472,2014 QL433,298.145886,0.0,0.000008,298.145887,15.350567,8268570668335894776,62406.11472


In [21]:
diatable[(diatable["ssObjectId"] == test_id)].value_counts("diaSourceId")

diaSourceId
-9112763465827438840    1
 2014190870401796341    1
 2956629072724944049    1
 2953752972822018602    1
 2905571607555473454    1
                       ..
-3557787637099240757    1
-3611971859248121084    1
-3616614797387747902    1
-3673733155341467608    1
 9049818896742252656    1
Name: count, Length: 326, dtype: int64

In [22]:
diatable[(diatable["ssObjectId"] == test_id) & (diatable["band"] == "r")].value_counts("diaSourceId")

diaSourceId
-8989137328016011290    1
 1805778899780713919    1
 4172232680473561059    1
 4078564031605990189    1
 3990456179953581022    1
                       ..
-3973452237660372181    1
-4294335785927851736    1
-4457341712489589716    1
-4684177232370434022    1
 8731362318873589325    1
Name: count, Length: 109, dtype: int64

In [23]:
# df_dict = {"dia":diatable,
#            "sss":sssource_table,
#            "sso":ssobject_table,
#            "mpc":mpcorb_table}

In [24]:
# for x in df_dict:

#     _df = df_dict[x].copy()[df_dict[x]["ssObjectId"]==test_id]
#     _df["ssObjectId"] = fake_id
#     df_dict[x] = pd.concat([df_dict[x],_df],axis = 0).reset_index(drop=True)
#     print(len(_df),len(df_dict[x]))

In [25]:
# diatable = df_dict["dia"]
# sssource_table = df_dict["sss"]
# ssobject_table = df_dict["sso"]
# mpcorb_table = df_dict["mpc"]

In [26]:
# _df = diatable.copy()[diatable["ssObjectId"]==test_id]
# _df["ssObjectId"] = fake_id
# diatable = pd.concat([diatable,_df],axis = 0).reset_index(drop=True)
# # print(len(_df),len(df_dict[x]))
# _df

In [27]:
# # add an outburst
# t0 = 63100
# t1 = 63600
# mag_shift = -1.5

# mask = ((diatable["ssObjectId"] == fake_id) &
#         (diatable["midPointMjdTai"] > t0) &
#         (diatable["midPointMjdTai"] < t1))

# diatable.loc[mask,"mag"] += mag_shift

In [28]:
diatable

,band,ccdVisitId,dec,decErr,decTrue,diaObjectId,diaSourceId,mag,magErr,magTrueVband,midPointMjdTai,nameTrue,ra,ra_dec_Cov,raErr,raTrue,snr,ssObjectId,ssObjectReassocTime
0,r,826978,-22.958176,0.000005,-22.958163,2905100135900213485,18188774906504369,22.122999,0.041,22.033504,61648.28255,2012 AA35,323.890681,0.0,0.000005,323.890684,22.464245,2150553186630,61648.28255
1,r,2035180,-14.553622,0.000018,-14.553617,6644478959102540493,87006313523607576,23.186001,0.176,23.277239,63793.11123,2012 AA35,220.572183,0.0,0.000018,220.572151,5.233148,2150553186630,63793.11123
2,i,538202,1.090052,0.000003,1.090046,-1814395187774575115,102043887374547575,21.809999,0.037,21.941309,61155.17906,2012 AA35,195.195758,0.0,0.000003,195.195759,24.892813,2150553186630,61155.17906
3,g,1994507,-14.133374,0.000004,-14.133380,-2764074387554066817,149948551075835426,22.665001,0.049,21.961533,63715.30357,2012 AA35,230.760251,0.0,0.000004,230.760246,18.796614,2150553186630,63715.30357
4,g,534253,0.589982,0.000003,0.589986,-2101259887024488987,186636841932196305,22.417000,0.028,21.665754,61145.18953,2012 AA35,197.511556,0.0,0.000003,197.511556,32.894073,2150553186630,61145.18953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,g,7799,-10.670760,0.000004,-10.670763,-874443130836536444,-1345853232552594655,22.945000,0.046,22.346977,60228.05606,2014 QL433,323.532914,0.0,0.000004,323.532914,20.022480,8268570668335894776,60228.05606
846,r,1090090,29.320428,0.000035,29.320468,-7987737159207644782,-1334792330958883646,23.068001,0.211,23.155615,62113.17449,2014 QL433,74.691787,0.0,0.000035,74.691838,4.365090,8268570668335894776,62113.17449
847,r,4492,-10.889507,0.000006,-10.889517,-5709489065863214846,-786563943595319518,22.143000,0.057,22.113560,60223.98877,2014 QL433,322.778901,0.0,0.000006,322.778904,16.158491,8268570668335894776,60223.98877
848,i,1237285,-18.625892,0.000008,-18.625884,-713349703496633846,-754458044377129801,21.313000,0.060,21.484858,62406.11472,2014 QL433,298.145886,0.0,0.000008,298.145887,15.350567,8268570668335894776,62406.11472


In [29]:
# diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)
diatable.to_sql("diaSource", con=cnx, if_exists="replace", index=False)

850

In [30]:
# sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)
sssource_table.to_sql("ssSource", con=cnx, if_exists="replace", index=False)

850

In [31]:
# ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)
ssobject_table.to_sql("ssObject", con=cnx, if_exists="replace", index=False)

5

In [32]:
# mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)
mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="replace", index=False)

5

In [33]:
_diatable = diatable.copy()[diatable["ssObjectId"] == test_id]
_diatable["ssObjectId"] = fake_id
_diatable["diaSourceId"] = _diatable["diaSourceId"] * 1000

# add an outburst
t0 = 63100
t1 = 63600
mag_shift = -1.5

mask = (
    (_diatable["ssObjectId"] == fake_id)
    & (_diatable["midPointMjdTai"] > t0)
    & (_diatable["midPointMjdTai"] < t1)
)

_diatable.loc[mask, "mag"] += mag_shift

_diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)

326

In [34]:
_diatable[diatable["band"] == "r"]

/tmp/ipykernel_52725/2339136216.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  _diatable[diatable["band"]=="r"]


,band,ccdVisitId,dec,decErr,decTrue,diaObjectId,diaSourceId,mag,magErr,magTrueVband,midPointMjdTai,nameTrue,ra,ra_dec_Cov,raErr,raTrue,snr,ssObjectId,ssObjectReassocTime
430,r,1559730,7.863010,0.000004,7.863015,-3580038114250342596,2887483772454859384,20.820000,0.022,20.833990,62949.16153,2007 YP19,144.570527,0.0,0.000004,144.570536,41.865185,6098332225018000,62949.16153
433,r,780833,-22.900968,0.000002,-22.900965,-521206056487428976,9031080621074865304,21.375999,0.014,21.343254,61586.24199,2007 YP19,270.687276,0.0,0.000002,270.687274,65.788147,6098332225018000,61586.24199
436,r,1525503,4.891761,0.000003,4.891765,-2089631523328994047,-9018027133072812392,20.544001,0.014,20.540991,62893.29190,2007 YP19,153.876449,0.0,0.000003,153.876452,65.788147,6098332225018000,62893.2919
437,r,1518294,4.769100,0.000002,4.769100,8562264045153953494,-8347653590176410624,20.754000,0.012,20.740877,62883.29148,2007 YP19,155.143920,0.0,0.000002,155.143925,76.752838,6098332225018000,62883.29148
439,r,803754,-22.478758,0.000004,-22.478761,-3629038990732461328,-2246078978940982088,21.924000,0.034,21.922724,61616.00915,2007 YP19,265.896019,0.0,0.000004,265.896015,27.089237,6098332225018000,61616.00915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,r,1578191,8.436433,0.000003,8.436428,-3020165854097591068,-6304576322577092128,21.493000,0.023,21.428377,62979.98330,2007 YP19,144.908269,0.0,0.000003,144.908267,40.044960,6098332225018000,62979.9833
747,r,14134,10.709536,0.000002,10.709537,-3499583069220269056,-758838375483899976,20.813999,0.012,20.796556,60236.10681,2007 YP19,9.406403,0.0,0.000002,9.406405,76.752838,6098332225018000,60236.10681
748,r,771981,-23.210095,0.000005,-23.210088,-3336099871633094386,-2618953178366439064,21.737000,0.034,21.680283,61556.35286,2007 YP19,276.470667,0.0,0.000005,276.470674,27.089237,6098332225018000,61556.35286
753,r,1546446,6.433843,0.000002,6.433845,-2893420366368583536,-3813093290538167840,20.277000,0.009,20.259291,62924.23636,2007 YP19,148.071518,0.0,0.000002,148.071521,102.337112,6098332225018000,62924.23636


In [35]:
_sssource_table = sssource_table.copy()[sssource_table["ssObjectId"] == test_id]
_sssource_table["ssObjectId"] = fake_id
_sssource_table["diaSourceId"] = _sssource_table["diaSourceId"] * 1000
_sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)

_ssobject_table = ssobject_table.copy()[ssobject_table["ssObjectId"] == test_id]
_ssobject_table["ssObjectId"] = fake_id
_ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)

_mpcorb_table = mpcorb_table.copy()[mpcorb_table["ssObjectId"] == test_id]
_mpcorb_table["ssObjectId"] = fake_id
_mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)

1

In [36]:
cnx.close()

Testing everything went correctly...

In [37]:
cnx = sqlite3.connect(db_fname)

In [38]:
# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId = {ssoid}
#                 """
# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId in {ssoid_list}
#                 """

example_query = f"""
                SELECT
                    ssObject.ssObjectId, ssSource.diaSourceId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
                    topocentricDist, heliocentricDist
                FROM
                    ssObject
                    JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
                    JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
                WHERE
                    ssObject.ssObjectId = {fake_id} and band = 'r'
                """

In [39]:
pd.read_sql_query(example_query, cnx)

,ssObjectId,diaSourceId,mag,magErr,band,midPointMjdTai,ra,dec,phaseAngle,topocentricDist,heliocentricDist
0,6098332225018000,7572962361378202184,22.760000,0.069,r,61469.38552,272.404111,-24.234828,15.537251,3.699083,3.521290
1,6098332225018000,5959460032317872944,22.643999,0.060,r,61473.39835,273.311348,-24.189850,15.804915,3.644592,3.524037
2,6098332225018000,316593712646882056,22.708000,0.114,r,61477.39732,274.162500,-24.140413,16.024870,3.589410,3.526729
3,6098332225018000,-8909479090051372512,22.575001,0.068,r,61485.38436,275.693450,-24.031717,16.314137,3.477325,3.531972
4,6098332225018000,-3412259241410896528,22.465000,0.042,r,61497.32416,277.520845,-23.855948,16.335381,3.307792,3.539473
...,...,...,...,...,...,...,...,...,...,...,...
104,6098332225018000,7264874908728208176,20.311001,0.013,r,60705.11646,109.793814,18.921026,7.616888,1.799676,2.739629
105,6098332225018000,3063975096637317000,20.304001,0.016,r,60706.12867,109.613810,18.928812,8.016209,1.804875,2.739833
106,6098332225018000,-2831084673266836416,20.698000,0.012,r,62469.15210,37.696298,19.070094,8.446361,1.985002,2.905847
107,6098332225018000,-4735777048316064960,20.289000,0.012,r,62445.22742,42.202635,21.014299,1.922925,1.945042,2.933355


In [40]:
cur = cnx.cursor()

In [41]:
res = cur.execute("SELECT * FROM sqlite_schema")

In [42]:
res.fetchall()

[('table',
  'diaSource',
  'diaSource',
  2,
  'CREATE TABLE "diaSource" (\n"band" TEXT,\n  "ccdVisitId" INTEGER,\n  "dec" REAL,\n  "decErr" REAL,\n  "decTrue" REAL,\n  "diaObjectId" INTEGER,\n  "diaSourceId" INTEGER,\n  "mag" REAL,\n  "magErr" REAL,\n  "magTrueVband" REAL,\n  "midPointMjdTai" REAL,\n  "nameTrue" TEXT,\n  "ra" REAL,\n  "ra_dec_Cov" REAL,\n  "raErr" REAL,\n  "raTrue" REAL,\n  "snr" REAL,\n  "ssObjectId" INTEGER,\n  "ssObjectReassocTime" TEXT\n)'),
 ('table',
  'ssSource',
  'ssSource',
  3,
  'CREATE TABLE "ssSource" (\n"diaSourceId" INTEGER,\n  "eclipticBeta" REAL,\n  "eclipticLambda" REAL,\n  "galacticB" REAL,\n  "galacticL" REAL,\n  "heliocentricDist" REAL,\n  "heliocentricVX" REAL,\n  "heliocentricVY" REAL,\n  "heliocentricVZ" REAL,\n  "heliocentricX" REAL,\n  "heliocentricY" REAL,\n  "heliocentricZ" REAL,\n  "mpcUniqueId" INTEGER,\n  "phaseAngle" REAL,\n  "predictedDecErr" REAL,\n  "predictedMagnitude" REAL,\n  "predictedMagnitudeErr" REAL,\n  "predictedRaDecCov" 